In [ ]:
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../data/labels.csv")
df = df[:10000]
df["label"] = df["label"].apply(lambda x: eval(x))

In [ ]:
Y = [i for i in df["label"]]
Y = np.array(Y)
index = 0
for i in Y:
    print(i)
    if index == 100:
        break
    index += 1

In [ ]:
def open_img(path):
    raw = Image.open(path)
    return np.array(raw)

In [ ]:
def showimage(img):
    plt.imshow(img)
    plt.show()

In [ ]:
X = np.zeros((0, 100, 100, 3), dtype=np.uint8)
for i in range(10000):
    X = np.concatenate((X, [open_img("../data/images/%d.jpeg" % i)]))

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers # type: ignore

In [ ]:
np.save("../X.npy", X)
np.save("../Y.npy", Y)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3, random_state=42069)

In [ ]:
def compile(model, epoch):
  # Compilation using adam optimization algorithm
  model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=["accuracy"]
  ) 
  history = model.fit(
    xtrain,
    ytrain,
    epochs=epoch,
    validation_data=(xtest, ytest)
  )
  return model, history

def setfigure():
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.ylim([0, 1])
  plt.legend(loc='lower right')
  plt.show()

def evaluate(history):
  plt.plot(history.history['accuracy'], label='accuracy')
  plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
  setfigure()

In [ ]:
model = tf.keras.models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Conv2D(256, (3, 3), activation='relu'),
    # layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(11, activation='sigmoid')
])
model, historytest = compile(model, 10)
evaluate(historytest)

In [ ]:
model = tf.keras.models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((3, 3)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.GlobalAveragePooling2D(),
    # tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(11, activation='sigmoid')
])
model.summary()
model, historytest = compile(model, 20)
evaluate(historytest)